## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Helper Functions

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray'
    please call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    
    return masked_image, mask

def separate_lines_into_left_right(lines, slope_thresh):
    """
    Separates line segments (lines) into left and right line segments
    based on the sign of slope of each line segment. 
    
    Only the slope values greater than 'slope_thresh' are retained by 
    getting rid of slope values not corresponding to lanes
    """
    
    left_lines = []
    right_lines = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            
            # Calculate slope
            if x2-x1 != 0:
                slope = (y2-y1)/(x2-x1)
            else:
                slope = 1000.0
                
            # Check slope against a threshold
            if abs(slope) > slope_thresh:
                # Separate lines into left and right based on the sign of slope
                if slope > 0:
                    right_lines.append(line)
                elif slope < 0:
                    left_lines.append(line)

    
    return left_lines, right_lines

def calc_robust_slope_and_intercept(lines):
    """
    Calculate the average segment by calculating the average values of each 
    co-ordinate. In other words, calculate the average of x1 values (x1_avg),
    x2 values (x2_avg), y1 values (y1_avg), and y2 values (y2_avg)
    
    Calculate the slope and intercept of the average segment formed by 
    (x1_avg, y1_avg) and (x2_avg, y2_avg)    
    """
    n = 1
    x1_avg = 0
    x2_avg = 0
    y1_avg = 0
    y2_avg = 0

    for line in lines:
         for x1,y1,x2,y2 in line:
            x1_avg = x1_avg + (x1 - x1_avg)/n
            x2_avg = x2_avg + (x2 - x2_avg)/n
            y1_avg = y1_avg + (y1 - y1_avg)/n
            y2_avg = y2_avg + (y2 - y2_avg)/n
            n += 1    
            
    [slope, intercept] = np.polyfit([x1_avg, x2_avg], [y1_avg, y2_avg], 1)
    
    return slope, intercept

def draw_lines_ext(img, lines, color=[255, 0, 0], thickness=12):
    """    
    Draws extrapolated line segments to map out the full extent of 
    the lane lines with `color` and `thickness` parameter. Lines are 
    drawn on the image inplace (mutates the image).
    
    Separates the line segments into left and right line segments
    based on the sign of their slopes.
    
    Calculates the average slope and average intercept of left 
    and right line segments.
    
    Find the start and end points on each side using the above
    average slope and intercept values, image size, and end point
    of region of interest polygon.
    
    Draw lines joining the start and end points on each side with
    given color and thickness. 
    
    """   
    # Separate line segments into left and right line segments
    slope_thresh = 0.5
    left_lines, right_lines = separate_lines_into_left_right(lines, slope_thresh)    
    
    # Calculate slope and intercept of left and right line segments 
    slope_left, intercept_left = calc_robust_slope_and_intercept(left_lines)    
    slope_right, intercept_right = calc_robust_slope_and_intercept(right_lines)
    
    # Find the start and end points on the left and the right sides
    y1 = img.shape[0] 
    y2 = img.shape[0] * tts_y # tts_y is the polygon_top_start for y-coordinate (row)
    
    right_x1 = (y1 - intercept_right) / slope_right
    right_x2 = (y2 - intercept_right) / slope_right

    left_x1 = (y1 - intercept_left) / slope_left
    left_x2 = (y2 - intercept_left) / slope_left

    # Convert calculated end points from float to int
    if ~np.isnan(right_x1) and ~np.isnan(right_x2) and ~np.isnan(left_x1) and ~np.isnan(left_x2):
        y1 = int(y1)
        y2 = int(y2)
        right_x1 = int(right_x1)
        right_x2 = int(right_x2)
        left_x1 = int(left_x1)
        left_x2 = int(left_x2)

        # Draw the right and left lines on image
        cv2.line(img, (right_x1, y1), (right_x2, y2), color, thickness)
        cv2.line(img, (left_x1, y1), (left_x2, y2), color, thickness)      
    
    return

def draw_lines(img, lines, color=[255, 0, 0], thickness=12):
    """  
    Draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).

    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
    return

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    # Calculate Hough lines using probabilistic Hough line transform
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    
    # Create a blank image of the same size as the input image
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    line_img_ext = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)    
    
    # Draw line segments on the blank image 
    draw_lines(line_img, lines) 
    # Draw extrapolated line segments on the blank image
    draw_lines_ext(line_img_ext, lines) 
    
    return line_img_ext, line_img

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [ ]:
import os
os.listdir("test_images/")

## Lane Detection Pipeline



In [ ]:
# Lane detection pipeline
plot_flag = 1
# Read in an image
img = mpimg.imread('test_images/whiteCarLaneSwitch.jpg')

# Print out some stats and plot the original image
print('This image is:', type(img), 'with dimensions:', img.shape)
if plot_flag == 1:
    plt.figure(); plt.imshow(img); plt.title('Original image')   
    mpimg.imsave("intermediate_images_output/original_image.png", img)

# Grayscale the image
gray = grayscale(img)
if plot_flag == 1:
    plt.figure(); plt.imshow(gray, cmap='gray'); plt.title('Gray scale image')
    mpimg.imsave("intermediate_images_output/grayscaled_image.png", gray, cmap='Greys_r')

# Define a kernel size and apply Gaussian smoothing
kernel_size = 5
blur_gray = gaussian_blur(gray, kernel_size)
if plot_flag == 1:
    plt.figure(); plt.imshow(blur_gray, cmap='gray'); plt.title('Gaussian blurred image')
    mpimg.imsave("intermediate_images_output/guassian_filtered_image.png", blur_gray, cmap='Greys_r')

# Define parameters for Canny edge detection and apply
low_threshold = 50
high_threshold = 150
edges = canny(blur_gray, low_threshold, high_threshold)
if plot_flag == 1:
    plt.figure(); plt.imshow(edges, cmap='gray'); plt.title('Canny edges')
    mpimg.imsave("intermediate_images_output/canny_edge_image.png", edges, cmap='Greys_r')

# Create a region of interest and use it to mask edges 
imshape = img.shape
tbs_x = 0.1 # polygon_bottom_start for x (column)
tts_x = 0.4 # polygon_top_start for x (column)
tte_x = 0.6 # polygon_top_end for x (column)
tbe_x = 0.9 # polygon_bottom_end for x (column)
tbs_y = 1   # polygon_bottom_start for y (row)
tts_y = 0.6 # polygon_top_start for y (row)
tte_y = 0.6 # polygon_top_end for y (row)
tbe_y = 1   # polygon_bottom_end for y (row)
vertices = np.array([[(tbs_x*imshape[1],tbs_y*imshape[0]),(tts_x*imshape[1],tts_y*imshape[0]), 
                      (tte_x*imshape[1],tte_y*imshape[0]), (tbe_x*imshape[1],tbe_y*imshape[0])]], dtype=np.int32)
masked_edges, mask = region_of_interest(edges, vertices)
if plot_flag == 1:
    plt.figure(); plt.imshow(mask, cmap='gray'); plt.title('Mask')
    mpimg.imsave("intermediate_images_output/region_mask.png", mask, cmap='gray')
    plt.figure(); plt.imshow(masked_edges, cmap='gray'); plt.title('Masked edges')
    mpimg.imsave("intermediate_images_output/masked_edges.png", masked_edges, cmap='Greys_r')

# Define the Hough transform parameters
rho = 1 # distance resolution in pixels of the Hough grid
theta = np.pi/180 # angular resolution in radians of the Hough grid
threshold = 30 # minimum number of votes (intersections in Hough grid cell)
min_line_len = 10 # minimum number of pixels making up a line
max_line_gap = 1 # maximum gap in pixels between connectable line segments

# Run Hough tranform on edge detected image
line_image_ext, line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)
if plot_flag == 1:
    plt.figure(); plt.imshow(line_image, cmap='gray'); plt.title('Line image')
    mpimg.imsave("intermediate_images_output/hough_lines.png", line_image)
    plt.figure(); plt.imshow(line_image_ext, cmap='gray'); plt.title('Line image with extrapolated lanes')
    mpimg.imsave("intermediate_images_output/hough_lines_ext.png", line_image_ext)

# Draw lines on the original image
lines_edges = weighted_img(line_image, img, α=0.8, β=1., γ=0.)
lines_edges_ext = weighted_img(line_image_ext, img, α=0.8, β=1., γ=0.)
if plot_flag == 1:
    plt.figure(); plt.imshow(lines_edges); plt.title('Original image with superimposed lines')
    mpimg.imsave("intermediate_images_output/detected_lanes.png", lines_edges)
    plt.figure(); plt.imshow(lines_edges_ext); plt.title('Original image with superimposed extrapolated lines')
    mpimg.imsave("intermediate_images_output/detected_lanes_ext.png", lines_edges_ext)   

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
# The process_image() function is called for each image in a video
def process_image(img):

    # Grayscale the image
    gray = grayscale(img)

    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = gaussian_blur(gray, kernel_size)

    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)

    # Create a region of interest and use it to mask edges 
    imshape = img.shape
    tbs_x = 0.1 # polygon_bottom_start for x (column)
    tts_x = 0.4 # polygon_top_start for x (column)
    tte_x = 0.6 # polygon_top_end for x (column)
    tbe_x = 0.9 # polygon_bottom_end for x (column)
    tbs_y = 1   # polygon_bottom_start for y (row)
    tts_y = 0.6 # polygon_top_start for y (row)
    tte_y = 0.6 # polygon_top_end for y (row)
    tbe_y = 1   # polygon_bottom_end for y (row)
    vertices = np.array([[(tbs_x*imshape[1],tbs_y*imshape[0]),(tts_x*imshape[1],tts_y*imshape[0]), 
                          (tte_x*imshape[1],tte_y*imshape[0]), (tbe_x*imshape[1],tbe_y*imshape[0])]], dtype=np.int32)
    masked_edges, mask = region_of_interest(edges, vertices)

    # Define the Hough transform parameters
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 30 # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 10 # minimum number of pixels making up a line
    max_line_gap = 1 # maximum gap in pixels between connectable line segments

    # Run Hough tranform on edge detected image
    line_image_ext, _ = hough_lines(masked_edges, rho, theta, threshold, min_line_len, max_line_gap)

    # Draw lines on the raw image
    lines_edges_ext = weighted_img(line_image_ext, img, α=0.8, β=1., γ=0.)
    
    return lines_edges_ext

### Video with the solid white lane on the right

In [ ]:
output = 'test_videos_output/solidWhiteRight_withDetectedLanes.mp4'
# original_clip = VideoFileClip('test_videos/solidWhiteRight.mp4').subclip(0,5) #subclip of the first 5 seconds
original_clip = VideoFileClip("test_videos/solidWhiteRight.mp4")
processed_clip = original_clip.fl_image(process_image) #NOTE: this function expects color images!!
%time processed_clip.write_videofile(output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))

### Video with the solid yellow lane on the left

In [ ]:
output = 'test_videos_output/solidYellowLeft_withDetectedLanes.mp4'
# original_clip = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5) #subclip of the first 5 seconds
original_clip = VideoFileClip("test_videos/solidYellowLeft.mp4")
processed_clip = original_clip.fl_image(process_image) #NOTE: this function expects color images!!
%time processed_clip.write_videofile(output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))

### Video with the challenging case

In [ ]:
output = 'test_videos_output/challenge_withDetectedLanes.mp4'
# original_clip = VideoFileClip('test_videos/challenge.mp4').subclip(0,5) #subclip of the first 5 seconds
original_clip = VideoFileClip("test_videos/challenge.mp4")
processed_clip = original_clip.fl_image(process_image) #NOTE: this function expects color images!!
%time processed_clip.write_videofile(output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))